<a href="https://colab.research.google.com/github/mattwantshouses/name_parsing/blob/main/Prod_Random_Goodies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# compare 2 sheets with highlighting

Use this to compare the output between two iterations of the same script.

The script will combine both into one sheet, and for every field will add two fields below - "self" and "other". It will highlight the right cell if different from the left cell.

##Instructions##
When prompted choose both files you want to compare.

Best Practice name your files with Version # and the earlier version will be on top in your downloads folder. Look at the printout to know which version is "self" and which is "other".

In [8]:
# version 3
# highlights cells red if different

import pandas as pd
import logging
from datetime import datetime
from google.colab import files
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.comments import Comment
from openpyxl.formatting.rule import CellIsRule
import pytz

# Section 1: Setup logging
logging.basicConfig(filename='comparison.log', level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

# Section 2: Helper function to load files
def load_file(file_path):
    try:
        if file_path.endswith('.csv'):
            data = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format")
        logging.info(f'Successfully loaded {file_path}')
        return data
    except Exception as e:
        logging.error(f'Error loading file {file_path}: {e}')
        raise

# Section 3: Upload and load files
def upload_files():
    uploaded = files.upload()
    if len(uploaded) == 2:
        file_paths = list(uploaded.keys())
        sheet1 = load_file(file_paths[0])
        sheet2 = load_file(file_paths[1])
        print(f"'self' is {file_paths[0]}")
        print(f"'other' is {file_paths[1]}")
    elif len(uploaded) == 1:
        file_path = list(uploaded.keys())[0]
        sheet1 = load_file(file_path)
        logging.info("Only one file uploaded. Upload the second file.")
        print("Upload the second file.")
        uploaded = files.upload()
        file_path = list(uploaded.keys())[0]
        sheet2 = load_file(file_path)
        print(f"'self' is {file_path}")
        print(f"'other' is the previously uploaded file")
    else:
        raise ValueError("Please upload one or two files.")
    return sheet1, sheet2

# Section 4: Compare DataFrames
def compare_sheets(sheet1, sheet2):
    try:
        comparison = sheet1.compare(sheet2, keep_shape=True, keep_equal=True)
        comparison.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in comparison.columns.values]
        comparison = comparison.reset_index()
        logging.info('Comparison successful')
        return comparison
    except Exception as e:
        logging.error(f'Error comparing sheets: {e}')
        raise

# Section 5: Highlight differences in the output and apply conditional formatting
def highlight_differences_and_apply_formatting(sheet1, sheet2, comparison, output_path):
    try:
        writer = pd.ExcelWriter(output_path, engine='openpyxl')
        comparison.to_excel(writer, sheet_name='Differences', index=False)
        writer.book.save(output_path)

        wb = load_workbook(output_path)
        ws = wb['Differences']
        red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')

        for row in range(2, ws.max_row + 1):
            for col in range(1, ws.max_column + 1):
                cell = ws.cell(row=row, column=col)
                if 'self' in str(cell.value) or 'other' in str(cell.value):
                    original_value = cell.value
                    cell.value = original_value.split(',')[0].replace('self', sheet1.columns[col - 1]).replace('other', sheet2.columns[col - 1])
                    cell.fill = red_fill
                    comment_text = f"{original_value.split(',')[0]}"
                    cell.comment = Comment(comment_text, "Comparison Script")

        for col in range(ws.max_column, 1, -2):
            col_letter = ws.cell(row=1, column=col).column_letter
            col_letter_left = ws.cell(row=1, column=col-1).column_letter
            rule = CellIsRule(operator='notEqual', formula=[f'{col_letter_left}2'], fill=red_fill)
            ws.conditional_formatting.add(f'{col_letter}2:{col_letter}{ws.max_row}', rule)

        wb.save(output_path)
        logging.info(f'Successfully highlighted differences and applied formatting in {output_path}')
        print(f"Comparison complete. Differences saved to '{output_path}' with highlights and conditional formatting.")
    except Exception as e:
        logging.error(f'Error highlighting differences: {e}')
        raise

# Section 6: Save the comparison results and trigger download
def save_and_download_comparison(comparison, sheet1, sheet2):
    try:
        # Get current time and convert to EST
        est = pytz.timezone('US/Eastern')
        current_time = datetime.now(est).strftime("%m-%d-%Y_%I-%M-%S %p")
        filename = f"Comparison - Foreclosure Results_{current_time}.xlsx"
        highlight_differences_and_apply_formatting(sheet1, sheet2, comparison, filename)
        files.download(filename)
    except Exception as e:
        logging.error(f'Error saving comparison: {e}')
        raise

# Main script execution
if __name__ == "__main__":
    try:
        logging.info('Script started')
        print("Upload the first file.")
        sheet1, sheet2 = upload_files()
        comparison = compare_sheets(sheet1, sheet2)
        save_and_download_comparison(comparison, sheet1, sheet2)
        logging.info('Script completed successfully')
    except Exception as e:
        logging.error(f'Script failed: {e}')
        print(f"Error: {e}")


Upload the first file.


Saving Parsed addresses Version 4 2-20pm.csv to Parsed addresses Version 4 2-20pm (7).csv
Saving Parsed addresses Version 5.xlsx to Parsed addresses Version 5 (7).xlsx
'self' is Parsed addresses Version 4 2-20pm (7).csv
'other' is Parsed addresses Version 5 (7).xlsx
Comparison complete. Differences saved to 'Comparison - Foreclosure Results_07-08-2024_05-16-21 PM.xlsx' with highlights and conditional formatting.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Combine Scraper Results + Foreclosure file
### Combine the base file (ex: Foreclosures) with the scraper file to retain all the info

This will link both files together based on the Parcel ID.
You will get all the results of the larger file (Foreclosure List) included, so you will need to delete the rows that do not have a match.

Both files must have a "Parcel ID" column. It doesn't have to be exact, but it does need to include those items.

In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import os
from google.colab import files
import io
from datetime import datetime
import pytz

In [ ]:
# Version 2 - only matches the records present in the smaller of the two files

# 1. Function to find the parcel ID column
def find_parcel_id_column(df):
    for col in df.columns:
        if 'parcel' in col.lower() and 'id' in col.lower():
            return col
    return None

# 2. Function to load a file (CSV or Excel)
def load_file(file_name, file_content):
    try:
        if file_name.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(file_content), dtype=str)
        elif file_name.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(io.BytesIO(file_content), dtype=str)
        else:
            raise ValueError(f"Unsupported file format for {file_name}. Please use CSV or Excel files.")
        return df
    except Exception as e:
        print(f"Error loading file {file_name}: {e}")
        return None

# 3. Main script
def main():
    print("Please upload both files (CSV or Excel):")
    uploaded_files = files.upload()

    if len(uploaded_files) != 2:
        print(f"Error: Expected 2 files, but {len(uploaded_files)} were uploaded. Please run the cell again and upload exactly 2 files.")
        return

    file_names = list(uploaded_files.keys())
    dfs = []

    for file_name, file_content in uploaded_files.items():
        df = load_file(file_name, file_content)
        if df is None:
            print(f"Error loading {file_name}. Exiting.")
            return
        dfs.append(df)

    # Determine which dataframe has fewer rows
    if len(dfs[0]) <= len(dfs[1]):
        df_left, df_right = dfs[0], dfs[1]
        file_name_left, file_name_right = file_names[0], file_names[1]
    else:
        df_left, df_right = dfs[1], dfs[0]
        file_name_left, file_name_right = file_names[1], file_names[0]

    # 4. Find parcel ID columns
    parcel_id_col_left = find_parcel_id_column(df_left)
    parcel_id_col_right = find_parcel_id_column(df_right)

    if parcel_id_col_left is None or parcel_id_col_right is None:
        print("Error: Unable to find a suitable parcel ID column in one or both files.")
        return

    print(f"Using '{parcel_id_col_left}' from {file_name_left} and '{parcel_id_col_right}' from {file_name_right} as parcel ID columns.")

    # 5. Merge dataframes
    try:
        merged_df = pd.merge(df_left, df_right, left_on=parcel_id_col_left, right_on=parcel_id_col_right, how='left', suffixes=('_1', '_2'))
    except Exception as e:
        print(f"Error merging dataframes: {e}")
        return

    # 6. Save merged file
    try:
        # Get current date and time in EST
        est = pytz.timezone('US/Eastern')
        current_time = datetime.now(est).strftime("%m-%d-%Y_%H-%M-%S")
        output_filename = f"Combined Base-Scraper Results_{current_time}.xlsx"

        # Save the DataFrame to an Excel file
        merged_df.to_excel(output_filename, index=False)

        # Download the file
        files.download(output_filename)

        print(f"Merged file '{output_filename}' has been created and is ready for download.")
        print("If the download doesn't start automatically, please check your browser's download settings.")

    except Exception as e:
        print(f"Error saving merged file: {e}")

if __name__ == "__main__":
    main()

Please upload both files (CSV or Excel):


Saving Foreclosures 07-08-24.csv to Foreclosures 07-08-24.csv
Saving khaliq foreclosure scraper results 07-08-24 (1).xlsx to khaliq foreclosure scraper results 07-08-24 (1).xlsx
Using 'parcel_id' from khaliq foreclosure scraper results 07-08-24 (1).xlsx and 'Parcel ID:' from Foreclosures 07-08-24.csv as parcel ID columns.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merged file 'Combined Base-Scraper Results_07-08-2024_23-27-48.xlsx' has been created and is ready for download.
If the download doesn't start automatically, please check your browser's download settings.
